# 深度学习 实验7

本节课我们实现高斯混合模型，并与聚类算法进行比较。

## 数据集

还是手写数字识别数据集（本节课仅需要训练集）

## Task 1

实现高斯混合模型与 Kmeans 聚类算法，方法不限，进行无监督学习，给出每个样本的预测结果。

## Task 2

分析两算法的预测结果与真实数据标签的关系，能否仅依靠这两个算法实现手写数字的分类？

数据读取：

首先从csv文件中将pixel取出来。然后进行了一次PCA降维，将每条数据的维度降到了300，因为用原来的784维数据来进行聚类的话，训练时间太久了，我这台机器的cpu属实背不起。

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA  # 进行主成分分析
train_data = pd.read_csv("train_data.csv")
x=train_data[[i for i in list(train_data.columns) if i != "label"]].values
x=np.array(x) / 255
pca = PCA(n_components=300)
pca.fit(x)
x_new = pca.transform(x)
x = x_new
frame = pd.DataFrame(x)
print(frame)
y=train_data["label"]


            0         1         2         3         4         5         6    \
0     -2.594493 -2.742397  0.718753  0.472986 -0.317968  1.919458 -2.680278   
1      6.672360 -1.413928 -1.967865  1.315387 -1.734819  2.895702  2.564217   
2     -3.478017 -1.152023  0.263354  0.306917 -1.857709 -1.268787  1.716859   
3     -0.650022  1.177187 -0.251551  2.979240 -1.669978  0.617217 -1.192546   
4      7.543960 -1.761385 -2.151424  0.739432 -2.555829  3.882603  2.213753   
...         ...       ...       ...       ...       ...       ...       ...   
41995  2.981785 -0.979820 -1.596046 -1.266134 -3.941721  2.181898 -1.067054   
41996 -4.156365 -1.185460  0.011534  0.151636 -1.411973 -0.591319  1.295334   
41997  1.318706  3.725550 -0.661198 -2.716396 -2.115152  0.114505  1.961913   
41998  1.283919  0.243200  2.735437  1.893374 -0.607113 -3.521515  1.577297   
41999 -1.428428  1.642718  0.246666  0.304237  0.215161  0.569244 -1.117873   

            7         8         9    ...       290 

GMM聚类

这里是直接调库的：在这里说一说自己对GMM的理解

首先GMM是将数据看成K个高斯分布的联合分布

然后计算每个样本点，在各个高斯分布的概率，选取概率最大的高斯分布视作这个样本点的簇

在每个样本点都分类结束后，更新每个簇的高斯分布的均值和协方差矩阵

不断迭代执行以上步骤，直到结果收敛或者达到迭代次数的限制

这个过程是EM算法的一种表现形式

这个过程也和KMeans算法类似：有分类和更新两步，只不过分类和更新的规则不同

In [3]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=10)
gmm.fit(x)

labels = gmm.predict(x)
frame_gmm = pd.DataFrame(x)
frame_gmm['cluster'] = labels
print(frame_gmm)

/Users/mac/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '


              0         1         2         3         4         5         6  \
0     -2.594493 -2.742397  0.718753  0.472986 -0.317968  1.919458 -2.680278   
1      6.672360 -1.413928 -1.967865  1.315387 -1.734819  2.895702  2.564217   
2     -3.478017 -1.152023  0.263354  0.306917 -1.857709 -1.268787  1.716859   
3     -0.650022  1.177187 -0.251551  2.979240 -1.669978  0.617217 -1.192546   
4      7.543960 -1.761385 -2.151424  0.739432 -2.555829  3.882603  2.213753   
...         ...       ...       ...       ...       ...       ...       ...   
41995  2.981785 -0.979820 -1.596046 -1.266134 -3.941721  2.181898 -1.067054   
41996 -4.156365 -1.185460  0.011534  0.151636 -1.411973 -0.591319  1.295334   
41997  1.318706  3.725550 -0.661198 -2.716396 -2.115152  0.114505  1.961913   
41998  1.283919  0.243200  2.735437  1.893374 -0.607113 -3.521515  1.577297   
41999 -1.428428  1.642718  0.246666  0.304237  0.215161  0.569244 -1.117873   

              7         8         9  ...       291 

KMeans聚类：

In [12]:
#training k-means model
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10)
kmeans.fit(x)
#predictions from kmeans
pred = kmeans.predict(x)
frame_kmeans = pd.DataFrame(x)
frame_kmeans['cluster'] = pred
print(frame_kmeans)

              0         1         2         3         4         5         6  \
0     -2.594493 -2.742397  0.718753  0.472986 -0.317968  1.919458 -2.680278   
1      6.672360 -1.413928 -1.967865  1.315387 -1.734819  2.895702  2.564217   
2     -3.478017 -1.152023  0.263354  0.306917 -1.857709 -1.268787  1.716859   
3     -0.650022  1.177187 -0.251551  2.979240 -1.669978  0.617217 -1.192546   
4      7.543960 -1.761385 -2.151424  0.739432 -2.555829  3.882603  2.213753   
...         ...       ...       ...       ...       ...       ...       ...   
41995  2.981785 -0.979820 -1.596046 -1.266134 -3.941721  2.181898 -1.067054   
41996 -4.156365 -1.185460  0.011534  0.151636 -1.411973 -0.591319  1.295334   
41997  1.318706  3.725550 -0.661198 -2.716396 -2.115152  0.114505  1.961913   
41998  1.283919  0.243200  2.735437  1.893374 -0.607113 -3.521515  1.577297   
41999 -1.428428  1.642718  0.246666  0.304237  0.215161  0.569244 -1.117873   

              7         8         9  ...       291 

聚类结果比较：

怎么实现聚类后的标签和真实的标签的对应？这里在网上找到了munkres算法来实现标签对应

将真实的标签和gmm聚类和kmeans聚类后的标签分别打印出来：

In [17]:
y_gmm=frame_gmm['cluster']
y_kmeans=frame_kmeans['cluster']
print(y)
print(y_gmm)
print(y_kmeans)

0        1
1        0
2        1
3        4
4        0
        ..
41995    0
41996    1
41997    7
41998    6
41999    9
Name: label, Length: 42000, dtype: int64
0        1
1        8
2        1
3        0
4        8
        ..
41995    8
41996    1
41997    6
41998    3
41999    0
Name: cluster, Length: 42000, dtype: int64
0        5
1        6
2        9
3        4
4        6
        ..
41995    6
41996    9
41997    1
41998    0
41999    4
Name: cluster, Length: 42000, dtype: int32


构造一个矩阵G：

G的维度为(真实标签的种类n)X(聚类标签的种类m)

$G_{ij}$为在真实标签中属于第i类且在聚类标签中属于第j类的个数

然后通过munkres实现标签对应，这里是直接调库，具体原理也不详细阐述，我是参考了：https://blog.csdn.net/hgh19951014/article/details/103496747

In [90]:
from munkres import Munkres
def label_map(y_real,y_cluster):
    G=np.zeros((10,10))
    for i in range(10):
        for j in range(10):
            for k in range(len(y_real)):
                if y_real[k]==i and y_cluster[k]==j:
                    G[i,j]+=1
    m=Munkres()
    index = m.compute(-G.T)
    index = np.array(index)
    c = index[:,1]
    y_cluster_map=np.zeros(len(y_cluster))
    for i in range(len(y_cluster)):
        y_cluster_map[i]=c[y_cluster[i]]
    return index,y_cluster_map

            
##误差计算函数  
def err_rate(y_real,y_cluster): 
    index,y_cluster_map=label_map(y_real,y_cluster)
    err_x=np.sum(y_real[:]!=y_cluster_map[:])
    missrate=err_x.astype(float)/len(y_cluster_map)
    return missrate,index
#标签对应输出
def labelmap(index):
    print("聚类标签->正式标签")
    for i in index:
        print(i[0],"->",i[1])

In [91]:
gmm_err_rate,gmm_index=err_rate(y,y_gmm)
kmeans_err_rate,kmeans_index=err_rate(y,y_kmeans)
print("GMM的标签对应")
labelmap(gmm_index)
print("GMM算法的误差率：",gmm_err_rate)
print("KMeans的标签对应")
labelmap(kmeans_index)
print("KMeans算法的误差率：",kmeans_err_rate)

GMM的标签对应
聚类标签->正式标签
0 -> 4
1 -> 1
2 -> 2
3 -> 6
4 -> 7
5 -> 3
6 -> 9
7 -> 8
8 -> 0
9 -> 5
GMM算法的误差率： 0.45716666666666667
KMeans的标签对应
聚类标签->正式标签
0 -> 6
1 -> 7
2 -> 3
3 -> 8
4 -> 4
5 -> 9
6 -> 0
7 -> 2
8 -> 5
9 -> 1
KMeans算法的误差率： 0.4642380952380952


有误差率可以看出两种聚类的误差率都将近一半，也就是说通过这两类聚类方法不能有效将图片向（0～9）聚类，因此不能按照这两个算法实现手写数字分类